<font color='red' size='6'>REMINDER</font>

# Fill in any place that says `YOUR CODE HERE`.
- You should remove the line that says `raise NotImplementedError()`. If you do not, your code will (unsurprisingly) throw a run-time error and cause everything to fail.
- Do **NOT** write your answer anywhere else other than where it says `YOUR CODE HERE`. Simply write your code directly below this comment in the **same code cell**.

# Make sure everything runs as expected.
- Go to the menubar, select *Kernel* > *Restart & Run All*

# Do <ins>NOT</ins> change the title (i.e., file name) of this notebook.

# Do <ins>NOT</ins> delete any of the cells in this notebook.

# Make sure you save your work
- Go to the menubar, select *File* > *Save and Checkpoint*

In [1]:
# Run this code cell
from nose.tools import assert_equal, assert_in, assert_is_instance
from nose.tools import assert_almost_equal, assert_not_equal, assert_not_in

# Setting

Pallet Movers utilizes a hub-and-spoke system similar to many LTL carriers. Their implementation requires a truck to leave a hub and visit all of its spokes to pick up freight and return back to the hub several times per week. They have asked you to provide the best route for a truck to leave the Charlotte hub, visit all four of its spokes, and return to Charlotte with the freight it picks up. They want you to use the total distance traveled as the metric to determine which route is best. 

# Traveling Salesman Problem (TSP)

This type of problem is called the traveling salesman problem (TSP) and is a classical operations research problem. The problem is simple to state but can be very difficult to solve to optimality, especially when the number of cities (e.g., spokes) is large. For $n$ cities, there are $(n-1)!$ possible tours. When $n$ is small, you could solve the problem with the brute-force approach of calculating the total distance traveled for all possible tours.

You are instead going to try to solve the problem using linear programming.



# An Incomplete Formulation

Here is a first attempt at a formulation for the TSP. We will soon see why it is "incomplete". 


$$
\begin{array}{lrcccl}
\max & \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} d_{ij}x_{ij} & & & \\
\textrm{subject to} & \sum\limits_{j\neq i}^{n}x_{ij} & = & 1 & \forall \textrm{   } i & \textrm{ leave each city once}\\
 & \sum\limits_{i\neq j}^{n} x_{ij} & = & 1 & \forall \textrm {   } j & \textrm{ enter each city once}\\
\textrm{where} &&&&& \\
\end{array} 
$$


$$
x_{ij} = 
    \left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if travel from city } i \text{ to city } j \\
    0, & \text{otherwise}
  \end{array}\right.
$$

$$
d_{ij} = \text{the distance between city } i \text{ and city } j
$$

What is missing from this formulation? Nothing says that you cannot have "subtours". For example, the solution with two subtours of: (1) Charlotte to Columbia to Fayetteville to Charlotte and (2) Greensboro to Spartanburg to Greensboro is feasible by these constraints. 

Your first task is to solve this problem using PuLP with the given formulation and **hope** we do not encounter subtours.

In [2]:
# import pulp as pl to solve the problem
import pulp as pl

# 1. Read in Distance Data

There is a file named `distances.csv` in the folder `data` that contains the distances between each of the five cities (the hub and the four spokes) for the Charlotte Hub. Read the data into a `DataFrame` variable named `distances`, making the first column of the file the index.

In [3]:
# YOUR CODE HERE
import pandas as pd

# read in the data
distances = pd.read_csv('./data/distances.csv', index_col=0)

In [4]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(distances.shape[0], 5, msg='Your DataFrame does not have the correct number of rows')
assert_equal(distances.shape[1], 5, msg='Your DataFrame does not have the correct number of columns')
assert_equal(type(distances['Charlotte']), pd.Series, msg='Did you use the first column as your index?')

# Data Structures
The next few tasks have you put the data into data structures that are easier to work with when using PuLP.

# 2. Create Distance List

It is easier for PuLP to use a list of distances rather than a `DataFrame`. Create a two-dimensional list from `distances` and store the results in a variable named `distance_list`. 

In [5]:
# YOUR CODE HERE
distance_list = [list(distances.iloc[i]) for i in range(len(distances))]

In [6]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_in('distance_list', dir(), msg='You did not name the variable `distance_list` as instructed')

In [7]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(type(distance_list), list, msg='Your variable distance_list is not a list')

<hr>

# 3. Create City Lists

1. Create a list named `from_cities` that gets all of the cities from the `distances` index.
2. Create a list named `to_cities` that gets all of the cities from the `distances` columns.

In [8]:
# YOUR CODE HERE
from_cities = list(distances.index)
to_cities = list(distances.columns)

In [9]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(type(from_cities), list, msg='Your variable from_cities is not a list')

In [10]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(type(to_cities), list, msg='Your variable to_cities is not a list')

<hr>

In [11]:
# This will create a dictionary that PuLP understands to hold
# the distances (miles) for each origin / destination pair
the_distances = pl.makeDict([from_cities, to_cities], distance_list, 999)

# Print it out to examine it
the_distances

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'Charlotte': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'Charlotte': 999,
                          'Greensboro': 93,
                          'Fayetteville': 138,
                          'Columbia': 93,
                          'Spartanburg': 75}),
             'Greensboro': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'Charlotte': 93,
                          'Greensboro': 999,
                          'Fayetteville': 95,
                          'Columbia': 183,
                          'Spartanburg': 161}),
             'Fayetteville': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'Charlotte': 138,
                          'Greensboro': 95,
                          'Fayetteville': 999,
                          'Columbia': 166,
           

In [12]:
# This list comprehension statement creates tuples
# containing (origin, destination) for all routes
routes = [(i, j) for i in from_cities for j in to_cities]

# print it out to examine it
routes

[('Charlotte', 'Charlotte'),
 ('Charlotte', 'Greensboro'),
 ('Charlotte', 'Fayetteville'),
 ('Charlotte', 'Columbia'),
 ('Charlotte', 'Spartanburg'),
 ('Greensboro', 'Charlotte'),
 ('Greensboro', 'Greensboro'),
 ('Greensboro', 'Fayetteville'),
 ('Greensboro', 'Columbia'),
 ('Greensboro', 'Spartanburg'),
 ('Fayetteville', 'Charlotte'),
 ('Fayetteville', 'Greensboro'),
 ('Fayetteville', 'Fayetteville'),
 ('Fayetteville', 'Columbia'),
 ('Fayetteville', 'Spartanburg'),
 ('Columbia', 'Charlotte'),
 ('Columbia', 'Greensboro'),
 ('Columbia', 'Fayetteville'),
 ('Columbia', 'Columbia'),
 ('Columbia', 'Spartanburg'),
 ('Spartanburg', 'Charlotte'),
 ('Spartanburg', 'Greensboro'),
 ('Spartanburg', 'Fayetteville'),
 ('Spartanburg', 'Columbia'),
 ('Spartanburg', 'Spartanburg')]

# 4. Create the Variables

There is a function called `.dicts()` on the class `LpVariable` that will easily create the 25 variables needed for this problem. The syntax is:

```python
pl.LpVariable.dicts('prefix', (list1, list2), lower_bound, upper_bound)
```

where `prefix` is what you want you variable name to be at the beginning - you should use the string 'route'; the tuple `(list1, list2)` are essentially $i$ and $j$ in the formulation above; the `lower_bound` should be a 0; and the `upper_bound` should be `None`. 

Create the variables and store the results in a variable named `vars`.

In [13]:
# YOUR CODE HERE
vars = pl.LpVariable.dicts('route', (from_cities, to_cities), 0, None)

In [14]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(type(vars), dict, msg='Your variable `vars` is not a dictionary')

In [15]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(len(vars), 5, msg='Your vars dictionary does not have correct number of entries')

<hr>

# 5. Build the Model

Create a variable named `model` that holds an instance of `LpProblem` that has the name "CLT_Spoke_Route" and is a minimization problem.

In [16]:
# YOUR CODE HERE
model = pl.LpProblem(name="CLT_Spoke_Route", sense=pl.LpMinimize)

In [17]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_in('model', dir(), msg='You did not name the variable `model` as instructed')

<hr>

# 6. Add Objective Function

The function `.lpSum()` calculates the sum of a list of linear expressions. Add the objective function to the `model` by sending in the appropriate list comprehension statement to `.lpSum()`.

In [18]:
# The objective function is added to 'model' first
# YOUR CODE HERE
model += pl.lpSum(vars[i][j] * the_distances[i][j] for i in from_cities for j in to_cities)

In [19]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(type(model.objective), pl.LpAffineExpression, msg='Your did not add the objective function correctly')

<hr>

# 7. Add the Constraints

1. Loop through the variable `from_cities` and add a constraint that requires that you can only leave the city once. Name each constraint with an f-string "Sum_of_Leaving_City_{i}" where $i$ is the city being left. For example, if the leaving city is Charlotte, the name of the constraint should be "Sum_of_Leaving_City_Charlotte".
2. Loop through the variable `to_cities` and add a constraint that requires that you can only enter the city once. Name each constraint with an f-string "Sum_of_Entering_City_{j}" where $j$ is the city being entered. For example, if the entering city is Spartanburg, the name of the constraint should be "Sum_of_Entering_City_Spartanburg".

In [20]:
# YOUR CODE HERE
for i in from_cities:
    model += pl.lpSum(vars[i][j] for j in to_cities if j != i) == 1, f'Sum_of_Leaving_City_{i}'

for j in to_cities:
    model += pl.lpSum(vars[i][j] for i in from_cities if i != j) == 1, f'Sum_of_Entering_City_{j}'

In [21]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(len(model.constraints), 10, msg='Your should have 10 constraints')

<hr>

In [22]:
# This will solve the model and print out the results
model.solve()

print(f'Model Status:{pl.LpStatus[model.status]}')
print(f'Objective = {pl.value(model.objective)}')

# Optimal solution
for v in model.variables():
    if(v.varValue > 0):
        print(v.name, '=', v.varValue)

Model Status:Optimal
Objective = 452.0
route_Charlotte_Spartanburg = 1.0
route_Columbia_Charlotte = 1.0
route_Fayetteville_Greensboro = 1.0
route_Greensboro_Fayetteville = 1.0
route_Spartanburg_Columbia = 1.0


# 8. Did You Find a Real Tour?

Did you find a real tour? Or were there sub-tours in your solution?

Create a variable named `all_done` and set it `True` if you have found an optimal solution with no sub-tours. Otherwise, set the variable to `False`.

In [23]:
# YOUR CODE HERE
all_done = pl.LpStatus[model.status] == pl.LpStatusOptimal

<hr>

# 9. Eliminate a Sub-Tour

You want to make sure that the sub-tour of Fayetteville to Greensboro to Fayetteville does not exist in your solution. To do so, you need to add the constraint that says the variables representing leaving Fayetteville and going Charlotte, Columbia, and Spartanburg plus the variables representing leaving Greensboro and going to Charlotte, Columbia, and Spartanburg must add to at least 1. This will force the elimination of the Fayetteville to Greensboro to Fayetteville sub-tour.

Add this sub-tour elimination constraint to `model` naming it "subtour_elimination_Fayetteville_Greensboro".

In [24]:
# YOUR CODE HERE
model += pl.lpSum(vars['Fayetteville'][j] for j in ['Charlotte', 'Columbia', 'Spartanburg']) + \
         pl.lpSum(vars['Greensboro'][j] for j in ['Charlotte', 'Columbia', 'Spartanburg']) >= 1, \
         'subtour_elimination_Fayetteville_Greensboro'

In [25]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(len(model.constraints), 11, msg='Your model should have 11 constraints now')

<hr>

In [26]:
# Solve the model again with the additional constraint
model.solve()

print(f'Model Status:{pl.LpStatus[model.status]}')
print(f'Objective = {pl.value(model.objective)}')

# Optimal solution
for v in model.variables():
    if(v.varValue > 0):
        print(v.name, '=', v.varValue)

Model Status:Optimal
Objective = 514.0
route_Charlotte_Fayetteville = 1.0
route_Columbia_Spartanburg = 1.0
route_Fayetteville_Greensboro = 1.0
route_Greensboro_Charlotte = 1.0
route_Spartanburg_Columbia = 1.0


<hr>

# 10. Eliminate Another Sub-Tour

You want to make sure that the sub-tour of Columbia to Spartanburg to Columbia does not exist in your solution. To do so, you need to add the constraint that says the variables representing leaving Columbia and going Charlotte, Fayetteville, and Greensboro plus the variables representing leaving Spartanburg and going to Charlotte, Fayetteville, and Greensboro must add to at least 1. This will force the elimination of the Columbia to Spartanburg to Columbia sub-tour.

Add this sub-tour elimination constraint to `model` naming it "subtour_elimination_Columbia_Spartanburg".

In [27]:
# YOUR CODE HERE
model += pl.lpSum(vars['Columbia'][j] for j in ['Charlotte', 'Fayetteville', 'Greensboro']) + \
         pl.lpSum(vars['Spartanburg'][j] for j in ['Charlotte', 'Fayetteville', 'Greensboro']) >= 1, \
         'subtour_elimination_Columbia_Spartanburg'

In [28]:
# This is a test cell
# If **NO** message is printed, it means that the tests passed
# One basic test to see if your code works
assert_equal(len(model.constraints), 12, msg='Your model should have 12 constraints now')

<hr>

In [29]:
# Solve the model again with additional constraint
model.solve()

print(f'Model Status:{pl.LpStatus[model.status]}')
print(f'Objective = {pl.value(model.objective)}')

# Optimal solution
for v in model.variables():
    if(v.varValue > 0):
        print(v.name, '=', v.varValue)

Model Status:Optimal
Objective = 523.0
route_Charlotte_Greensboro = 1.0
route_Columbia_Spartanburg = 1.0
route_Fayetteville_Columbia = 1.0
route_Greensboro_Fayetteville = 1.0
route_Spartanburg_Charlotte = 1.0


<hr>

## Additional Resources

[Case studies in PuLP's documentation][1]

----
[1]: https://coin-or.github.io/pulp/CaseStudies/

**&copy; 2022 - Present: Matthew D. Dean, Ph.D.   
Clinical Associate Professor of Business Analytics at William \& Mary.**